## 수능특강 확통 - 1단원 여러 가지 순열 - Level 3 - 3번 문제

$a_1 \sim a_8$이 있다. $a_i \in \{0,1,2\}$이고
$\displaystyle \sum_{k=1}^{8} a_k = 8$을 만족하는
$(a_1, a_2, ... , a_8)$의 순서쌍의 개수는?


접근 방법:

1. $2$가 $0$개인 경우, $1$개인 경우, ... $4$개인 경우 합
2. $(1+x+x^2)$에서 $x^8$의 계수

### 1\. $2$가 $0$개인 경우, $1$개인 경우, ... $4$개인 경우 합

$2$를 $k$개 고르면 나머지 $8-k$개 중에서 $1$을 $8-2k$개 골라야 한다.

$8$개 중에서 $2$를 $k$개 고르는 경우의 수: $_8C_k$

$8-k$개 중에서 $1$을 $8-2k$개 고르는 경우의 수: $_{8-k}C_{8-2k}$

총 경우의 수

$\displaystyle \sum^4_{k=0} {_8}C_{k}\cdot_{8-k}C_{8-2k} = \sum^4_{k=0} {{8!}\over{(8-2k)!k!k!}}$


In [1]:
import math

print(sum(math.comb(8, k) * math.comb(8 - k, 8 - 2 * k) for k in range(5)))


1107


### 2\. $(1+x+x^2)$에서 $x^8$의 계수

1. 직접 전개하기
2. 인수분해, 이항정리 적용
3. FFT 알고리즘 응용


### 2-1 직접 전개하기

파스칼의 삼각형과 유사하다. 윗줄의 3개 값의 합을 적는다.
[파스칼의 피라미드](https://en.wikipedia.org/wiki/Pascal%27s_pyramid#Sum_of_coefficients_of_a_layer_by_columns)와 관련이 있다.

$${1}$$
$${1\ \ 1\ \ 1}$$
$${1\ \ 2\ \ 3\ \ 2\ \ 1}$$
$${1\ \ 3\ \ 6\ \ 7\ \ 6\ \ 3\ \ 1}$$
$${1\ \ 4\ 10\ 16\ 19\ 16\ 10\ \ 4\ \ 1}$$

$0$부터 세는 것으로 하면 $i$줄 $j$번째 값은 $(1+x+x^2)^{i}$에서 $x^{j}$의 계수이다.

예를 들어 $2$번째 줄은 $(1+x+x^2)^{2}=1x^0+2x^1+3x^2+2x^3+1x^4$을 나타내기 때문에 ${1\ 2\ 3\ 2\ 1}$이다.

따라서 8째줄의 정 가운데 값을 찾으면 된다.

처음 풀 때 이 방법을 썼는데, 계산을 단순화하기 위해 2번째 줄과 6번째 줄을 구하여 다음과 같이 계산하였다.

$$\sum^4_{k=0} f(2, k) \cdot f(6, 8-k)$$

또한 표의 대칭성을 이용하여 계산을 반으로 줄일 수 있다.

분할 정복법을 이용하면 2의 거듭제곱 행만 계산하여 연산량을 더 줄일 수 있다.

또한 FFT를 이용하면 다항식의 곱셈을 빠르게 계산할 수 있는데, 이는 2-3에서 논의한다.


In [2]:
# 전체 표 구하기

table = [[0] * (2 * i + 1) for i in range(9)]
table[0][0] = 1

for i in range(8):
    for j in range(2 * i + 1):
        table[i + 1][j] += table[i][j]
        table[i + 1][j + 1] += table[i][j]
        table[i + 1][j + 2] += table[i][j]

print(table[8][8])


1107


In [3]:
# 2째 줄, 6째 줄 이용

s = 0
for i in range(5):
    s += table[2][i] * table[6][8 - i]
print(s)


1107


In [4]:
# 2의 거듭제곱 이용


def polynomial_multiply(p1: list[float], p2: list[float]):
    # 곱하는 두 다항식이 같으면 곱셈을 대략 절반으로 줄일 수 있지만, 구현은 생략하였다.
    l1 = len(p1)
    l2 = len(p2)
    p3 = [0] * (l1 + l2 - 1)
    for i in range(l1):
        for j in range(l2):
            p3[i + j] += p1[i] * p2[j]
    return p3


def polynomial_pow(base: list[float], exponent: int, p: list[float] = [1]):
    if exponent & 1:
        p = polynomial_multiply(p, base)
    exponent >>= 1
    if exponent == 0:
        return p
    base = polynomial_multiply(base, base)
    return polynomial_pow(base, exponent, p)


def main():
    p = [1, 1, 1]
    print(polynomial_pow(p, 8)[8])


main()


1107


### 2-2 인수분해, 이항정리 적용

$1+x+x^2$을 인수분해하여 1차식의 곱으로 나타내고, 이항정리와 근과 계수의 관계를 이용하여 계산할 수 있다.

$1+x+x^2 = (x+z)(x+\overline{z})$라고 하자. $1+x+x^2$의 두 허근은 각각 $-z, -\overline{z}$이다.

근과 계수의 관계에 따라 $z + \overline{z} = 1, z\overline{z}=1$이 성립한다.

또, $1+x+x^2=0$에서 양변에 $(x-1)$을 곱해 정리하면 $x^3=1$.
그러므로 $z^3 = \overline{z}^3 = -1$이다.

$(1+x+x^2)^{8}$ = $(x+z)^{8}(x+\overline{z})^{8}$로 쓸 수 있다.

$x^8$의 계수를 구해야 하기 때문에
$(x+z)^8$에서 $x$를 $k$개 선택하고, $(x+\overline{z})^8$에서 $x$를 $8-k$개 선택하면 된다.

따라서 $x^8$의 계수는 다음과 같다.

$\displaystyle \sum^8_{k=0} ({_8}C{_k}\cdot{z^{8-k}})\cdot({_{8}}C_{8-k}\cdot{\overline{z}^{k}})$
$\displaystyle =\sum^8_{k=0} {_8}C{_k}{_{8}}C_{8-k}{z^{8-k}}{\overline{z}^{k}}$
$\displaystyle =\sum^8_{k=0} {_8}C{_k}^2{z^{8-k}}{\overline{z}^{k}}$

켤레쌍끼리 더해지도록 식을 조작하면

$\displaystyle \sum^3_{k=0} {_8}C{_k}^2({z^{8-k}}{\overline{z}^{k}}+{z^{k}}{\overline{z}^{8-k}}) + {_8}C{_4}^2{z^{4}}{\overline{z}^{4}}$

$z\overline{z}=1$을 대입하면

$\displaystyle \sum^3_{k=0} {_8}C{_k}^2({z^{8-2k}}+{\overline{z}^{8-2k}}) + {_8}C{_4}^2$

$z$와 $\overline{z}$의 거듭제곱의 주기성을 살펴보면 다음과 같은 규칙을 찾을 수 있다.

$z^n + \overline{z}^n = {\begin{cases} \phantom{-}2&{n\equiv 0 \pmod{6}} \\ \phantom{-}1&{n\equiv 1, 5 \pmod{6}} \\ -1&{n\equiv 2, 4 \pmod{6}} \\ -2&{n\equiv 3 \pmod{6}} \end{cases}}$


$k = 0$부터 대입하여 전개하면 다음과 같다.

$-{_8C_0}^2 + 2{_8C_1}^2 - {_8C_2}^2 - {_8C_3}^2 + {_8C_4}^2$


In [5]:
import math


def f(k):
    # 8Ck의 거듭제곱 구하는 함수
    return math.comb(8, k) ** 2


print(-f(0) + 2 * f(1) - f(2) - f(3) + f(4))


1107


### 2-3 FFT 알고리즘 응용

FFT의 아이디어로 다항식의 값으로 계수를 구하는 방법이다.

$f(x) = (1+x+x^2)^8$일 때, $16$차 다항식 $f$에서 $x^8$의 계수를 찾으면 된다.

문제를 단순화하여 먼저 $2$차 다항식에서 $1$차항의 계수를 구해보자.

$f(x) = ax^2 + bx + c$라고 하자.

$f(1) = a + b + c, f(-1) = a - b + c$

따라서 $\displaystyle b = {f(1) - f(-1)\over2}$

중요한 아이디어는 2번 곱해 1이 되는 수를 대입해보는 것이다.

비슷한 방법으로 $4$차 다항식에서 $2$차항의 계수를 구할 땐 4번 곱해 1이 되는 $1, -1, i, -i$를 대입해본다. 

$\displaystyle f(x) = a_4x^4 + a_3x^3 + a_2x^2 + a_1x^1 + a_0x^0$

$\displaystyle f(1) = a_4 + a_3 + a_2 + a_1 + a_0, $
$\displaystyle f(-1) = a_4 - a_3 + a_2 - a_1 + a_0, $

$\displaystyle f(i) = a_4 - {a_3}\cdot{i} - a_2 + {a_1}\cdot{i} + a_0, $
$\displaystyle f(-i) = a_4 + {a_3}\cdot{i} - a_2 - {a_1}\cdot{i} + a_0$

$\displaystyle a_2 = {\{f(1) + f(-1)\} - \{f(i) + f(-i)\} \over 4}$

일반적으로 $n-1$차 다항식 $f$에서 다음이 성립한다. 자세한 증명은 생략하겠다.

$\displaystyle let \; f(x) = \sum^n_{k=0} a_kx^k$

$\displaystyle a_m = {1 \over n}{\sum^{n-1}_{k=0} f(e^{2 \pi i {k \over n}})e^{-2 \pi i {mk \over n}}}$

간단히 설명하자면 $m$차항에서는 $({e^{2 \pi i {k \over n}}})^m$과 $e^{-2 \pi i {mk \over n}}$의 곱이 항상 1이 되지만,
다른 항들은 복소수 거듭제곱의 주기성 때문에 합이 0이 된다. 

$f(x) = (1+x+x^2)^8, n=16, m=8$을 대입하자.

16차 다항식이지만 16차항과 상수항을 합쳐 하나로 생각할 수 있고 이때 8차항은 영향을 받지 않는다.

$\displaystyle a_8 = {1 \over 16}{\sum^{15}_{k=0} f(e^{2 \pi i {k \over 16}})e^{-2 \pi i {8k \over 16}}}$

$\displaystyle \phantom{a_8} = {1 \over 16}{\sum^{15}_{k=0} f(e^{k \pi i \over 8})e^{-k \pi i}}$

$ {k \pi \over 8}  = \theta$ 라고 하자

$(1 + e^{i\theta} + e^{2i\theta}) = 1 + \cos\theta + \cos{2\theta} + (\sin\theta + \sin{2\theta})i$

$\phantom{(1 + e^{i\theta} + e^{2i\theta})} = (1 + \cos{2\theta}) + \cos\theta + (\sin\theta + \sin{2\theta})i$

$\phantom{(1 + e^{i\theta} + e^{2i\theta})} = 2\cos^2\theta + \cos\theta + (\sin\theta + 2\sin\theta\cos\theta)i$

$\phantom{(1 + e^{i\theta} + e^{2i\theta})} = (2\cos\theta + 1) (\cos\theta + i\sin\theta)$

$\phantom{(1 + e^{i\theta} + e^{2i\theta})} = (2\cos\theta + 1) e^{i\theta}$

$\displaystyle \phantom{=} {1 \over 16}{\sum^{15}_{k=0} f(e^{k \pi i \over 8})e^{-k \pi i}}$

$\displaystyle = {1 \over 16}{\sum^{15}_{k=0} \{(1 + 2\cos{k \pi \over 8}) \cdot e^{k i \pi \over 8}\}^8 e^{-k \pi i}}$

$\displaystyle = {1 \over 16}{\sum^{15}_{k=0} (1 + 2\cos{k \pi \over 8})^8 e^{k i \pi} e^{-k \pi i}}$

$\displaystyle = {1 \over 16}{\sum^{15}_{k=0} (1 + 2\cos{k \pi \over 8})^8}$

In [6]:
import math

s = 0
for k in range(16):
    s += (1+2*math.cos(k*math.pi/8)) ** 8

print(s/16)

1106.9999999999995


부동소수점 연산 오차를 고려하면 계산 결과가 답과 일치한다. 

꼭 $15$차 다항식의 FFT를 활용할 필요는 없고, $8$차 이상이면 된다.

$15$차 다항식으로 놓은 이유는 항의 개수가 $2$의 거듭제곱이기 때문에 여기에서 FFT 알고리즘이 연상되었다.

In [7]:
import math

# n-1차 다항식에서 FFT 활용
n = 9
m = 8

s = 0
for k in range(n):
    s += (1+2*math.cos(k*2*math.pi/n)) ** m

print(round(s/n))

1107


위 코드에서 $n>m$일 때, $(1+x+x^2)^m$에서 $x^m$의 계수가 출력된다.

이 방법이 계산 과정은 훨씬 복잡하지만, 정수만 사용하는 경우의 수 문제에서 복소수의 주기성을 활용한다는 점이 아름답게 느껴졌다. 



$\displaystyle \phantom{=} {1 \over 16}{\sum^{15}_{k=0} (1 + 2\cos{k \pi \over 8})^8}$

$\displaystyle \phantom{=} {1 \over 16}{\sum^{7}_{k=0} \{(1 + 2\cos{k \pi \over 8})^8 + (1 - 2\cos{k \pi \over 8})^8\}}$